# Binary image processing

Author: Julian Lißner

For questions and feedback write a mail to: [lissner@mib.uni-stuttgart.de](mailto:lissner@mib.uni-stuttgart.de)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys

sys.path.extend( ['provided_functions', 'incomplete_functions' ] )
import image_operations as operate
import binary_image_processing as binary
import result_check as check
import image_kernels as kernels
from image_conversion import load_grayscale

## Erosion and dilation
- have a straight forward implementation for binary images (special case!)
- dilation places the kernel as a stencil on every foreground pixel it finds <br>
$\quad\blacktriangleright$ every value larger 0 is kept after convolution
- erosion erodes every foreground pixel but those where the kernel fully fits <br>
$\quad\blacktriangleright$ every value which is not the sum of the kernel is omitted
- erosion is the counterpart to dilation, but they are not invertible
- both dilation and erosion can be implemented with a convolution followed by a segmentation
- the major difference is the segmentation threshold

-------
__Task:__ Implement the `erosion` and `dilation` function in 'binary_image_processing.py'.

In [ ]:
check.erosion_dilation( binary.erosion)

In [ ]:
check.erosion_dilation( binary.dilation)

In [ ]:
image = np.load( 'data/convolution_check.npz')['arr_0']
kernel = kernels.disc( 15)
dilated = binary.erosion( image, kernel)
eroded = binary.dilation( image, kernel)
recovered = binary.dilation( eroded, kernel) 

fig, axes = plt.subplots( 2,2, figsize=(12,12))
axes = axes.flatten()
axes[0].imshow( image )
axes[1].imshow( dilated )
axes[2].imshow( eroded )
axes[3].imshow( recovered)

titles =[ 'original_image', 'dilation', 'erosion', 'recovered image']
for ax in axes:
    ax.set_title( titles.pop(0))
    ax.axis( 'off')

- erosion is the counterpart to dilation
- neither erosion, nor dilation are invertible
- the above deployed process is called opening
- closing is first a dilation, followed by erosion <br>
$\quad \blacktriangleright$ small gaps in the foreground are _closed_
- opening is first an erosion, followed by a dilation <br>
$\quad \blacktriangleright$ background is _opened up_
-----
__Task:__ Implement the `opening` and `closing` functions in 'binary_image_processing.py'.

In [ ]:
if not np.allclose( recovered, binary.opening(image, kernel) ):
    raise Exception( 'opening most likely wrongly implemented')

- opening and closing can be used to remove salt and pepper noise
- Salt noise: 'foreground pixels' where there should be background
- pepper noise: 'background pixels where they should be foreground
- a combination of salt and pepper noise requires multiple processing steps

------
__Task:__ Deploy a chain of opening and closing with the correct kernels to restore the rectangle in the image.

In [ ]:
images = list( np.load( 'data/opening_closing_images.npz').values() )
#images[0] -> original image
glitched_image = images[3]

#kernel = kernels.#TODO
recovered_image = glitched_image #TODO
#TODO....

fig, axes = plt.subplots( 1,4, figsize=(15,6) )
axes[0].imshow( images[0])
axes[1].imshow( glitched_image)
axes[2].imshow( recovered_image) 
axes[3].imshow( np.roll( images[0], [-2,-2], axis=[0,1]) - recovered_image) #images[0] does not exactly match the solution
titles = ['desired image', 'glitched image', 'recovered image', 'deviation']
for ax in axes:
    ax.set_title( titles.pop(0))
for ax in axes[1:]:
    ax.axis( 'off')


------------
-----------
## Restoring ancient texts
- before doing involved image processing, the image has to be cropped
- cropping of images can be easily done in python with images being represented as arrays $\blacktriangleright$ slicing


In [ ]:
## Function definition for better readable code
def show_images( image_1, image_2, axis='on'):
    """
    This is a minor function for pure convenience in the lower code blocks
    """
    fig, axes = plt.subplots( 1,2, figsize=(12,12) )
    axes[0].imshow( image_1, cmap='gray' )
    axes[1].imshow( image_2, cmap='gray')
    
    titles = [ 'Previous step', 'Current step']
    for ax in axes:
        ax.set_title( titles.pop(0 ))
        ax.axis( axis)
    return fig, axes


(Interactive plotting has been moved to a python script because it functions differently in jupyter notebooks for different workstations)

---------

__Task:__ Crop the image to ommit the (non paper) background.<br>
Optional: Write an interactive plot function for instantaneous visual feedback. A template is given in 'provided_functions/interactive_plot_example.py'.

In [ ]:
image_orig = 255-load_grayscale( 'figures/kassenzettel.jpeg' )

image = image_orig#[ #TODO] # Hint: Crop the image here by trial-and-visualization 
               
show_images( image_orig, image )
cropped_image = image.copy() #used for further (repeatable) processing

- grayscale images are usually given with color values from 0-255, i.e. [0, 1, 2, ...255]
- binary segmentation segments the image to two values, black and white
- for e.g. old text it is usually beneficial to deal with segmented black white (binary) images
- the metric of (human) vision is a very good error indicator

----
__Task:__ Choose a good segmentation threshold to nicely recover the ancient texts.<br>

In [ ]:

image = operate.#TODO( cropped_image, #TODO

show_images( cropped_image, image)
segmented_image = image.copy()


- the image can be manually edited by overwriting specific values via indexing
- this is especially simple for binary images, obtained from e.g. segmentation 
- if you have any residual black regions, you can overwrite them with indexing
----------
__Task:__ OPTIONAL, set remaining black regions to be white by direct indexing.

In [ ]:
image = segmented_image.copy()
#image[ #TODO ] #TODO

show_images( segmented_image, image )
fixed_image = image.copy()

- the remaining lines can be seen as pepper noise
- depending on the kernel opening/closing can be rather coarse
- text is also rather thin, so be careful when choosing your kernel
- Note, here black is the 'background'

--------
__Task__: Create yourself a kernel (of 0 and 1) to remove the 'pepper noise'

In [ ]:
kernel = np.array( [[#TODO] ]) 
image = #TODO
    
show_images( fixed_image, image)


__Task:__ admire your result.

In [ ]:
show_images( image_orig, image, axis='off')